In [68]:
import sys
import argparse
import configparser
from pathlib import Path

import logging

# see this for inspiration https://github.com/szymonlipinski/examples/blob/master/python_settings/fixed.py

In [164]:
logging.basicConfig(level=logging.DEBUG, format='%(name)s:%(funcName)s %(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

In [165]:
class file():
    '''class that creates a pathlib.Path().expanduser().resolve() object from a string
    Args:
        file(`str`): string representation of a file path'''
    def __init__(self, file):
        self.file = file
        
    @property
    def file(self):
        return self._file
    
    @file.setter
    def file(self, file):
        f = Path(file).expanduser().resolve()
        if f.exists():
            self._file = f
            self.parent = f.parent
            self.exists = True
        else:
            logging.warning(f'file does not exist: {f}')
            self._file = None
            self.parent = None
            self.exists = False
            
    def __repr__(self) -> Path:
        return repr(str(self.file))
    
    def __str__(self):
        return(str(self.file))

In [145]:
class Options():
    '''parse command line options
    Args:
        args(`list`): sys.argv is typically passed here
    Properties:
        parser(`argparse.ArgumentParser`): argument parser object
        args(`list`): list of arguments
        options(NameSpace): argument parser generated namespace of arguments
        opts_dict(`dict`): namespace -> dictionary'''
    def __init__(self, args):
        self.parser = argparse.ArgumentParser()      
        self.args = args
    
    @property
    def parser(self):
        '''The argparser object'''
        return self._parser
    
    @parser.setter
    def parser(self, parser):
        if parser:
            self._parser = parser
    
    @property
    def options(self):
        '''argparser namespace of the parsed arguments'''
        try:
            return self._options
        except AttributeError as e:
            self._parse_args()
            return self._options

    
    @options.setter
    def options(self, options):
        if options:
            self._options = options
#             self.opts_dict = vars(self.options)
        else:
            self._options = None
    
    @property
    def opts_dict(self):
        '''namespace of dictionary of parsed options'''
        self._parse_args()
        self._opts_dict = vars(self.options)
        return self._opts_dict
        
    def _parse_args(self):
        '''parse known arguments and discard unknown arguments'''
        options, unknown = self.parser.parse_known_args()
        logging.info(f'discarding unknwon commandline arguments: {unknown}')
        self.options = options
    
    def add_argument(self, *args, **kwargs):
        '''add arguments to the parser using standard argparse.ArgumentParser
        Args:
            *args, **kwargs'''
        try:
            self.parser.add_argument(*args, **kwargs)
        except argparse.ArgumentError as e:
            logging.warning(f'failed adding conflicting option {e}')

In [155]:
class ConfigFile():
    def __init__(self, cfg_file=None):
        self.cfg_file = cfg_file
    
    @property
    def cfg_file(self):
        '''file() class object for storing resolved file path

        Args:
            cfg_file(`str` or `pathlib.Path`): path to .INI type configuration file

        Sets:
            config(`configparser.ConfigParser`): configuration parser object
            config_dict(`dict`): dictionary rpresentation of configuration file'''        
        return self._cfg_file
    
    @cfg_file.setter
    def cfg_file(self, cfg_file):
        self._cfg_file = file(cfg_file)
        # must use the acutal pathlib.Path() object NOT The __str__ or __repr__ functions
        # from file class
        self.config = self.cfg_file.file

    @property
    def config(self):
        '''configparser.ConfigParser object for managing configuration file
        Args:
            cfg_file(`str` or `pathlib.Path`): configuration file to parse
        Sets:
            config(`configparser.ConfigParser`): configuration parser object
            config_dict(`dict`): dictionary rpresentation of configuration file'''            
        return self._config
    
    @config.setter
    def config(self, cfg_file):
        config = configparser.ConfigParser()
        config.read(cfg_file)
        self._config = config
        self.config_dict = self._config_2dict(config)
    
    
        
    def _config_2dict(self, configuration):
        '''convert an argparse object into a dictionary

        Args:
            configuration(`configparser.ConfigParser`)

        Returns:
            `dict`'''
        d = {}
        for section in configuration.sections():
            d[section] = {}
            for opt in configuration.options(section):
                d[section][opt] = configuration.get(section, opt)

        return d

In [234]:
d = {'foo': 'bar', 'spam': {'snake': 7, 'bear': [9, 9, 9], 'cow': {'name': 'sam', 'age': 4}}}
u = {'foo': 'foobar', 'spam': {'bear': [1, 4, 6, 6]}}

In [238]:
'foo' in d.keys()

True

In [232]:
print(d)

{'foo': 'bar', 'spam': {'snake': 7, 'bear': [9, 9, 9], 'cow': {'name': 'sam', 'age': 4}}}


In [233]:
d['foo'] = {}
print(d)

{'foo': {}, 'spam': {'snake': 7, 'bear': [9, 9, 9], 'cow': {'name': 'sam', 'age': 4}}}


In [192]:
def merge_config(default_cfg: dict, user_cfg: dict):
    merged_cfg = {}
    merged_cfg.update(default_cfg)
    merged_cfg.update(user_cfg)
    
    return merged_cfg


In [198]:
import collections
def dict_merge(d_one: dict, d_two: dict):
    '''Recursively merge d_one and d_two
        key/value pairs in d_two overwrite d_one
        Thanks: https://gist.github.com/angstwad/bf22d1822c38a92ec0a9#file-dict_merge-py
        
    Args:
        d_one(`dict`): dictionary one
        d_two(`dict`): dictionary two
        
    Returns:
        `dict`: merge of d_one and d_two where d_two overwrites d_one
    
    '''
    for k, v in d_two.items():
        if (k in d_one and isinstance(d_one[k], dict)
                and isinstance(d_two[k], collections.Mapping)):
            dict_merge(d_one[k], d_two[k])
        else:
            d_one[k] = d_two[k]
    return d_one

In [199]:
dict_merge(d, u)

{'foo': 'foobar',
 'spam': {'snake': 7, 'bear': [1, 4, 6, 6], 'cow': {'name': 'sam', 'age': 4}}}

In [178]:
merge_config(d, u)

{'foo': 'foobar', 'spam': {'bear': [1, 4, 6, 6]}}

In [156]:
c = ConfigFile(cfg_file='./slimpi.cfg')

In [157]:
c.config.sections()

['main', 'lms_server', 'layouts', 'logging']

In [158]:
c.config_dict

{'main': {'splash_screen': 'True'},
 'lms_server': {'host': '', 'port': '9000', 'player_id': ''},
 'layouts': {'display': 'epd5in83',
  'now_playing': 'threeRow',
  'stopped': 'clock',
  'splash': 'splash'},
 'logging': {'log_level': 'INFO'}}

In [214]:
o = Options(sys.argv)

In [215]:
o.add_argument('-l', '--log-level', dest='logging__log_level', action='store', 
               default='WARNING', choices=['DEBUG', 'INFO', 'WARNING', 'ERROR'])

In [216]:
o.add_argument('-p', dest='main__pop', action='store_true', default=True, help='stuff and things', )

In [227]:
vars(o.options)

{'logging__log_level': 'WARNING', 'main__pop': True}

In [226]:
for key in vars(o.options):
    match = re.search('(\w+)__(\w+)', key)
    print(match.group(1))
    print(match.group(2))

logging
log_level
main
pop


In [240]:
import re
def _nested_opts_dict(opts):
    d = {}
    options = vars(opts)
    for key in options.keys():
        match = re.search('(\w+)__(\w+)', key)
        section = match.group(1)
        option = match.group(2)
        if key not in d.keys():
            d[key] = {}
        
        d[key][option] = options[key]
    return d

In [241]:
_nested_opts_dict(o)

AttributeError: 'NoneType' object has no attribute 'group'

In [218]:
o.opts_dict

root:_parse_args INFO: discarding unknwon commandline arguments: ['-f', '/Users/aciuffo/Library/Jupyter/runtime/kernel-4a95dcca-2ceb-451b-b15b-d174c64a3ce8.json']


{'logging__log_level': 'WARNING', 'main__pop': True}

create a "Default" configuration object from the builtin config
Create a "user" configuration based on the user config
merge the default and the user overriding the default with the user version
merge the command line over the top of everything

finally creating a dictionary of options